In [10]:
import os

# define the directories where the imgaes are

pizza_directory="./pizza/"
not_pizza_directory="./not_pizza/"

# get the images from the directories

not_pizza_files=[[0, not_pizza_directory, i] for i in os.listdir(not_pizza_directory)]
pizza_files=[[1, pizza_directory, i] for i in os.listdir(pizza_directory)]

print("Number of pizza images:",len(pizza_files))
print("Number of non-pizza images:",len(not_pizza_files))

# combine the image sets

all_files = not_pizza_files + pizza_files

print("Total number of images:", len(all_files))

# each element in all_files is of the format [is_pizza: boolean, directory: string, filename: string]
# in other words, for each `img` in `all_files`:
#    - img[0] = 0 if not pizza, 1 if pizza
#    - img[1] = directory where the image is
#    - img[2] = file name

Number of pizza images: 983
Number of non-pizza images: 983
Total number of images: 1966


## Making all the images the same size and storing in new directory

Also storing everything into resizedImages array

In [16]:
from PIL import Image

standardSize = (512,512)

resizedImages = [[]]
# structure is:
# resizedImages[i][0] = image data
# resizedImages[i][0] = isPizza


PATH = os.path.abspath(os.getcwd())

dirExists = os.path.exists(f'{PATH}/resizedImages')

if (not dirExists):
    os.mkdir('resizedImages')

for i, image in enumerate(all_files):
    imgData = Image.open(image[1] + image[2])
    imgData = imgData.resize(standardSize)
    PATH
    imgData.save(f'{PATH}/resizedImages/{i}.jpg')
    # storing images
    resizedImages.append([imgData, image[0]])

# ! Takes about 30 seconds...